In [76]:
import pandas as pd
from datetime import datetime

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
# A. load data without unnecessary data
raw_input = pd.read_csv('PL_all_seasons.csv', \
usecols=['Date','HomeTeam','AwayTeam','FTR','B365H','B365D','B365A'])

# B. convert date from string to datetimes
raw_input['Date'] = pd.to_datetime(raw_input['Date'], dayfirst = True)

# append seasons
# Add a new column named 'season' 
Type_new = pd.Series([])

# running a for loop and asigning some values to series 
for i in range(len(raw_input)): 
    if raw_input["Date"][i] < datetime(2010, 8, 10): 
        Type_new[i]="09/10"
  
    elif raw_input["Date"][i] < datetime(2011, 8, 10): 
        Type_new[i]="10/11"
  
    elif raw_input["Date"][i] < datetime(2012, 8, 10):  
        Type_new[i]="11/12"
  
    elif raw_input["Date"][i] < datetime(2013, 8, 10):  
        Type_new[i]="12/13"
  
    elif raw_input["Date"][i] < datetime(2014, 8, 10):  
        Type_new[i]="13/14"
    
    elif raw_input["Date"][i] < datetime(2015, 8, 10):  
        Type_new[i]="14/15"
  
    elif raw_input["Date"][i] < datetime(2016, 8, 10):  
        Type_new[i]="15/16"

    elif raw_input["Date"][i] < datetime(2017, 8, 10):  
        Type_new[i]="16/17"
  
    elif raw_input["Date"][i] < datetime(2018, 8, 10):  
        Type_new[i]="17/18"
  
    elif raw_input["Date"][i] < datetime(2019, 8, 10):  
        Type_new[i]="18/19"

    else: 
        season[i]= raw_input["Date"][i] 
  
        
        # inserting new column with values of list made above         
raw_input.insert(2, "Season", Type_new) 
# raw_input.head(10)
#####################################################################################################################

# Combine total data frame

df = pd.DataFrame(raw_input)

In [78]:
# d. add boolean of result Home, Draw, Away and then print df

df['H_true'] = df['FTR'].str.count("H")
df['D_true'] = df['FTR'].str.count("D")
df['A_true'] = df['FTR'].str.count("A")

# e. multiply bolean of Draw (D_true) with B365 odds of draw to get each games return for you

df['return_H'] = ( df.loc[:, 'H_true'] * df.loc[:, 'B365H'] )
df['return_D'] = ( df.loc[:, 'D_true'] * df.loc[:, 'B365D'] )
df['return_A'] = ( df.loc[:, 'A_true'] * df.loc[:, 'B365A'] )

print(df)

# f. sum these individual returns to get you full season return

return_H = df[df.Season == "09/10"].return_H.sum()

print(return_H)


sum_return_D = return_D.sum()
print(sum_return_D)

#g count entries
No_of_matches = df.Date.count()
print(No_of_matches)

#h. find the net return by subtracting the DKK 1 * No of matches = DKK x you have betted during the season to find your gain/loss

net_return_D = sum_return_D - No_of_matches

print(net_return_D)

#i. find the return in percentage by dividing you gain with the DKK 1 * No of matches = DKK x you have betted during the season and subtract 1.

net_pct_returm_D = sum_return_D / No_of_matches - 1

print(net_pct_returm_D)



Date     HomeTeam Season      AwayTeam FTR  B365H  B365D  B365A  \
0    2009-08-15  Aston Villa  09/10         Wigan   A   1.67   3.60   5.50   
1    2009-08-15    Blackburn  09/10      Man City   A   3.60   3.25   2.10   
2    2009-08-15       Bolton  09/10    Sunderland   A   2.25   3.25   3.25   
3    2009-08-15      Chelsea  09/10          Hull   H   1.17   6.50  21.00   
4    2009-08-15      Everton  09/10       Arsenal   A   3.20   3.25   2.30   
...         ...          ...    ...           ...  ..    ...    ...    ...   
3795 2019-05-12    Liverpool  18/19        Wolves   H   1.30   6.00  11.00   
3796 2019-05-12   Man United  18/19       Cardiff   A   1.28   6.50  11.00   
3797 2019-05-12  Southampton  18/19  Huddersfield   D   1.44   4.75   8.50   
3798 2019-05-12    Tottenham  18/19       Everton   D   2.20   3.50   3.50   
3799 2019-05-12      Watford  18/19      West Ham   A   2.25   3.75   3.20   

      H_true  D_true  A_true  return_H  return_D  return_A  
0          0 

ValueError: No axis named 09/10 for object type <class 'pandas.core.frame.DataFrame'>

3578.21
3800
-221.78999999999996
-0.058365789473684204


Div      Date     HomeTeam      AwayTeam  FTHG  FTAG FTR  HTHG  HTAG HTR  \
0     E0  15/08/09  Aston Villa         Wigan     0     2   A     0     1   A   
1     E0  15/08/09    Blackburn      Man City     0     2   A     0     1   A   
2     E0  15/08/09       Bolton    Sunderland     0     1   A     0     1   A   
3     E0  15/08/09      Chelsea          Hull     2     1   H     1     1   D   
4     E0  15/08/09      Everton       Arsenal     1     6   A     0     3   A   
...   ..       ...          ...           ...   ...   ...  ..   ...   ...  ..   
3795  E0  12/05/19    Liverpool        Wolves     2     0   H     1     0   H   
3796  E0  12/05/19   Man United       Cardiff     0     2   A     0     1   A   
3797  E0  12/05/19  Southampton  Huddersfield     1     1   D     1     0   H   
3798  E0  12/05/19    Tottenham       Everton     2     2   D     1     0   H   
3799  E0  12/05/19      Watford      West Ham     1     4   A     0     2   A   

      ... BbMx>2.5  BbAv>2.5  Bb